In [1]:
%load_ext autoreload

%autoreload 1

In [2]:
import random
import numpy as np
import pandas as pd
from collections import Counter
from copy import deepcopy
import sys

sys.path.append('../protosp03/recommendation/')
sys.path.append('../protosp03/data/synthetic/')
%aimport matchings
%aimport market
%aimport upskillings
%aimport make_synthetic
%aimport recommendation

In [3]:
path = "../data/taxonomy/taxonomy_V4.csv"
mastery_levels = [1, 2, 3, 4]
years = [i for i in range(2023, 2017, -1)]

In [4]:
skills, learners, jobs, courses = make_synthetic.get_job_market(path, mastery_levels, years )

In [5]:
skill_supply, skill_demand, skill_trends, skills_attractiveness = market.get_all_market_metrics(skills, mastery_levels, learners, jobs, years)

In [33]:
learner = deepcopy(learners[80])
up_skilling_advice = up_skilling_market_advice(learner, skills, skills_attractiveness)
up_skilling_advice

('Management der ICT', 4)

In [30]:
enrollable_courses = matchings.get_all_enrollable_courses(learner, courses, threshold=0.5)

In [36]:
for course in enrollable_courses:
    if ('Führung in Konfliktsituationen', 4) in course["provided_skills"].items():
        print(course)

{'required_skills': {'Service Engineering': 1}, 'provided_skills': {'Führung in Konfliktsituationen': 4, 'Aufnahmefähigkeit': 1}}


In [41]:
recommendation.get_course_recommendation(learner, enrollable_courses, up_skilling_advice)

{'required_skills': {'Service Engineering': 4,
  'Zusammenarbeit in virtuellen Teams': 2},
 'provided_skills': {'Network Engineering': 3}}

In [47]:
def get_course_recommendation(learner, enrollable_courses, up_skilling_advice):
    """Get the course recommendation for a learner and a given up-skilling advice.

    Args:
        learner (dict): Learner's profile including possessed skills and levels.
        enrollable_courses (list): List of all enrollable courses.
        up_skilling_advice (tuple): A tuple of the skill and level to up-skill to.

    Returns:
        dict: The course recommendation
    """
    course_recommendation = None
    matching = 0
    for course in enrollable_courses:
        if up_skilling_advice in course["provided_skills"].items():
            print("Course", course["provided_skills"], "is recommended.")
            tmp_matching = matchings.learner_course_matching(learner, course)
            if tmp_matching > matching:
                matching = tmp_matching
                course_recommendation = course
    return course_recommendation

In [49]:
get_course_recommendation(learner, enrollable_courses, ('Network Engineering', 3))

Course {'Network Engineering': 3} is recommended.


{'required_skills': {'Service Engineering': 4,
  'Zusammenarbeit in virtuellen Teams': 2},
 'provided_skills': {'Network Engineering': 3}}